In [12]:
import pandas as pd
import numpy as np
import json

In [13]:
with open("cgam_parsed.json", "r") as f:
    result = json.load(f)


dfs_ebs = {}
for key in ["components", "connections"]:
    dfs_ebs[key] = pd.DataFrame.from_dict(result[key], orient="index").sort_index()

with open("cgam_tespy.json", "r") as f:
    result = json.load(f)


dfs_tespy = {}
for key in ["components", "connections"]:
    dfs_tespy[key] = pd.DataFrame.from_dict(result[key], orient="index").sort_index()

In [14]:
dfs_ebs["connections"][["e_T", "e_M"]] = dfs_ebs["connections"][["e_T", "e_M"]].round(6)
dfs_tespy["connections"][["e_T", "e_M"]] = dfs_tespy["connections"][["e_T", "e_M"]].round(6)

In [15]:
dfs_ebs["connections"]["m"]

1           90.956080
10           1.644320
2           90.956080
3           90.956080
4           92.600401
5           92.600401
6           92.600401
6P          92.600401
7           92.600401
8           14.000000
8P          14.000000
9           14.000000
E1                NaN
Logik             NaN
Logik_1           NaN
Logik_2           NaN
Logik_3           NaN
Rauchgas     1.000000
W1                NaN
W2                NaN
Name: m, dtype: float64

In [16]:
columns = ["m", "p", "T", "e_T", "e_M"]

overlapping_index = list(set(dfs_tespy[key].index.tolist()) & set(dfs_ebs[key].index.tolist()))
diff_to_tespy = (
    (
        dfs_ebs[key].loc[overlapping_index, columns]
        - dfs_tespy[key].loc[overlapping_index, columns]
    ) / dfs_tespy[key].loc[overlapping_index, columns]
).abs().replace(np.inf, np.nan)
diff_to_ebs = (
    (
        dfs_ebs[key].loc[overlapping_index, columns]
        - dfs_tespy[key].loc[overlapping_index, columns]
    ).abs() / dfs_tespy[key].loc[overlapping_index, columns]
).abs().replace(np.inf, np.nan)

In [17]:
diff_to_ebs

,m,p,T,e_T,e_M
1,0.000183,0.000000e+00,0.000000e+00,NaN,NaN
7,0.000155,0.000000e+00,1.002291e-03,4.902781e-03,NaN
10,0.001398,0.000000e+00,6.539424e-14,NaN,0.007728
5,0.000155,2.680162e-04,3.328763e-04,3.284637e-04,0.003172
2,0.000183,2.298427e-16,8.233857e-04,5.170441e-03,0.000541
6,0.000155,2.961505e-04,7.014391e-04,1.462352e-03,0.005671
6P,0.000155,1.769711e-04,8.883634e-04,2.936085e-03,0.006993
8P,0.000000,0.000000e+00,1.570503e-12,1.921363e-11,0.000000
9,0.000000,0.000000e+00,2.822459e-10,2.908972e-10,0.000000
4,0.000155,3.554894e-05,6.580088e-08,8.278086e-04,0.000539


In [18]:
diff_to_ebs[diff_to_ebs > 1e-3].dropna(how="all")

,m,p,T,e_T,e_M
7,NaN,NaN,0.001002,0.004903,NaN
10,0.001398,NaN,NaN,NaN,0.007728
5,NaN,NaN,NaN,NaN,0.003172
2,NaN,NaN,NaN,0.005170,NaN
6,NaN,NaN,NaN,0.001462,0.005671
6P,NaN,NaN,NaN,0.002936,0.006993
3,NaN,NaN,NaN,0.001337,NaN
